<a href="https://colab.research.google.com/github/JAEJAE93/fashion_product_classification/blob/main/fashion_classification_with_HuggingFace_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

In [2]:
!nvidia-smi

Wed Mar 31 06:48:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Google drive 접근
# http://blog.naver.com/PostView.nhn?blogId=skk1991&logNo=221746903289

from google.colab import drive

drive.mount('/content/gdrive/')
%cd gdrive/MyDrive/'Colab Notebooks'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks


In [4]:
import pickle
import numpy as np
import pandas as pd
import re
import os
import sys
import random
import time
import datetime

In [5]:
data_path = 'fashion_data/'
pkl_path = data_path + 'fashion_data.pkl'

# pkl load
with open(pkl_path, "rb") as f:
  total_data = pickle.load(f)

print("total_data: ", total_data.shape)
print("product category: ", (total_data["상품카테고리"].unique()))
total_data

total_data:  (31614, 3)
product category:  ['지갑' '스웨터' '점퍼/아우터' '코트' '자켓' '팬츠' '신발' '티셔츠' '가방' '트렌치 코트' '모자' '드레스'
 '머플러/마스크' '홀더 및 케이스' '양말' '스카프' '블라우스' '셔츠' '넥타이' '베스트' '스커트' '장갑' '벨트'
 '주얼리 및 타이핀']


,상품설명,상품카테고리,상품컬러
0,블랙 가죽 골드 퍼피로고 카드지갑,지갑,BLACK
1,[인기상품 재입고] 오렌지 19FW 리버서블 풀오버 니트,스웨터,NORMAL ORANGE
2,[셀럽 협찬 주X욱 착용] 머스타드 《가먼트다잉》 실키 후드점퍼,점퍼/아우터,LIGHT YELLOW
3,그레이 짜임패턴 울캐시미어혼방 니트,스웨터,LIGHT GREY
4,[LE RIRE] 베이지 단색 울캐시미어혼방 니트,스웨터,NORMAL BEIGE
...,...,...,...
31609,[ILFORD]블랙 엠보 가죽 베이직 그립백,가방,BLACK
31610,[HUDSON]블랙 가죽 H금속 로고 클러치백(M),가방,BLACK
31611,[LINDBERG] 블랙 화이트 로고 모던 클러치,지갑,BLACK
31612,블랙 위빙 가죽 장지갑,지갑,BLACK


In [6]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [7]:
def _preprocessing(text):
    """
        input: string
        output: string
    """
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'  # E-mail제거
    text = re.sub(pattern = pattern, repl = '', string = text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL제거
    text = re.sub(pattern = pattern, repl = '', string = text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern = pattern, repl = '', string = text)
    pattern = '<[^>]*>'  # HTML 태그 제거
    text = re.sub(pattern = pattern, repl = '', string = text)
    pattern = '[^\w\s]'  # 특수기호제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)  
    text = ' '.join(text.split()) # 2개 이상의 공백 제거
    text = text.lower()
    text = text.strip()
    return text

In [8]:
text_data = total_data[["상품설명", "상품카테고리"]]

# label encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(text_data["상품카테고리"])
text_data["상품카테고리"] = label_encoder.transform(text_data["상품카테고리"])

# text preprocessing
text_data["상품설명"] = [_preprocessing(x) for x in text_data["상품설명"]]
text_data = text_data.reset_index(drop=True)
text_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,상품설명,상품카테고리
0,블랙 가죽 골드 퍼피로고 카드지갑,18
1,인기상품 재입고 오렌지 19fw 리버서블 풀오버 니트,9
2,셀럽 협찬 주x욱 착용 머스타드 가먼트다잉 실키 후드점퍼,16
3,그레이 짜임패턴 울캐시미어혼방 니트,9
4,le rire 베이지 단색 울캐시미어혼방 니트,9
...,...,...
31609,ilford 블랙 엠보 가죽 베이직 그립백,0
31610,hudson 블랙 가죽 h금속 로고 클러치백 m,0
31611,lindberg 블랙 화이트 로고 모던 클러치,18
31612,블랙 위빙 가죽 장지갑,18


In [9]:
product_category = text_data["상품카테고리"].unique()
print("product_category num: ", len(product_category))
print("product_category: ", product_category)

train, test = train_test_split(text_data, test_size=0.25, random_state=42)
print("train: ", train.shape)
print("test: ", test.shape)
train

product_category num:  24
product_category:  [18  9 16 19 14 22 12 21  0 20  4  2  3 23 13 10  7  8  1  5 11 15  6 17]
train:  (23710, 2)
test:  (7904, 2)


,상품설명,상품카테고리
4610,그린 컬러배색 모피 퍼코트,16
21025,mmlg knit bucket black,4
10645,air line a1 블랙 4way 백팩,0
22341,네이비 포켓 울 니트베스트,9
30799,argyle fringe knit muffler_cream,3
...,...,...
29802,lord nermal ls gold,21
5390,블랙 가죽 로고 클러치백,0
860,그린 꼬르소 이지 스웻,21
15795,stripe pattern 네이비 버티컬 스트라이프 울 홑겹 베스트,9


# 전처리 - 훈련셋

In [10]:
# BERT 입력 형식에 맞게 변환
def _get_bert_sentences(sentences):
    sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

    return sentences

# 라벨 추출
def _get_label(targets, column):
    labels = targets[column].values

    return labels

# BERT의 토크나이저로 문장을 토큰으로 분리
def _get_tokenized_texts(version, sentences):
    tokenizer = BertTokenizer.from_pretrained(version, do_lower_case=False)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    return tokenizer, tokenized_texts

# 토큰화된 문장 숫자로 변경
def _get_input_ids(MAX_LEN, tokenizer, tokenized_texts): # MAX_LEN = 입력 토큰의 최대 시퀀스 길이
    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    return input_ids

# attention mask 얻기
def _get_attention_masks(input_ids):
    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    return attention_masks

# 데이터를 파이토치의 텐서로 변환
def _get_torch_tensor(input_ids, labels, attention_masks):
    test_inputs = torch.tensor(input_ids)
    test_labels = torch.tensor(labels)
    test_masks = torch.tensor(attention_masks)

    return test_inputs, test_labels, test_masks

![대체 텍스트](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

BERT의 입력은 위의 그림과 같은 형식입니다. Classification을 뜻하는 [CLS] 심볼이 제일 앞에 삽입됩니다. 파인튜닝시 출력에서 이 위치의 값을 사용하여 분류를 합니다. [SEP]은 Seperation을 가리키는데, 두 문장을 구분하는 역할을 합니다. 이 예제에서는 문장이 하나이므로 [SEP]도 하나만 넣습니다.
<br>
<br>
<br>

In [11]:
# BERT 입력 형식에 맞게 변환
sentences = _get_bert_sentences(train["상품설명"])
sentences[:10]

['[CLS] 그린 컬러배색 모피 퍼코트 [SEP]',
 '[CLS] mmlg knit bucket black [SEP]',
 '[CLS] air line a1 블랙 4way 백팩 [SEP]',
 '[CLS] 네이비 포켓 울 니트베스트 [SEP]',
 '[CLS] argyle fringe knit muffler_cream [SEP]',
 '[CLS] 블랙 차이나넥 언발란스 자켓 [SEP]',
 '[CLS] 카멜 폴리 혼방 부클 롱 자켓 [SEP]',
 '[CLS] 무지 숏 기모 맨투맨 블랙 엠오엠씨 [SEP]',
 '[CLS] 크림 부클내피 탈부착 면혼방 후드캐주얼점퍼 [SEP]',
 '[CLS] 화이트 라인배색 면 롱셔츠 [SEP]']

In [12]:
# 라벨 추출
labels = _get_label(train, '상품카테고리')
labels

array([16,  4,  0, ..., 21,  9, 19])

In [13]:
# BERT의 토크나이저로 문장을 토큰으로 분리

tokenizer, tokenized_texts = _get_tokenized_texts('bert-base-multilingual-cased', sentences)

print (sentences[0])
print (tokenized_texts[0])

[CLS] 그린 컬러배색 모피 퍼코트 [SEP]
['[CLS]', '그', '##린', '컬', '##러', '##배', '##색', '모', '##피', '퍼', '##코', '##트', '[SEP]']


In [14]:
input_ids = _get_input_ids(128, tokenizer, tokenized_texts)
input_ids[0]

array([  101,  8924, 27654,  9801, 30873, 76036, 41442,  9283, 97146,
        9913, 25517, 15184,   102,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [15]:
# 어텐션 마스크 얻기

attention_masks = _get_attention_masks(input_ids)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [16]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=42, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
# train
train_inputs, train_labels, train_masks = _get_torch_tensor(train_inputs, train_labels, train_masks)
# validation
validation_inputs, validation_labels, validation_masks = _get_torch_tensor(validation_inputs, validation_labels, validation_masks)

print("train_inputs: ", train_inputs.shape)
print("train_labels: ", train_labels.shape)
print("train_masks: ", train_masks.shape)
print()
print("validation_inputs: ", validation_inputs.shape)
print("validation_labels: ", validation_labels.shape)
print("validation_masks: ", validation_masks.shape)

train_inputs:  torch.Size([21339, 128])
train_labels:  torch.Size([21339])
train_masks:  torch.Size([21339, 128])

validation_inputs:  torch.Size([2371, 128])
validation_labels:  torch.Size([2371])
validation_masks:  torch.Size([2371, 128])


In [17]:
train_inputs

tensor([[  101,  8924, 56645,  ...,     0,     0,     0],
        [  101,  9638, 83616,  ...,     0,     0,     0],
        [  101,  9376, 17342,  ...,     0,     0,     0],
        ...,
        [  101, 18922, 23661,  ...,     0,     0,     0],
        [  101,  9376, 17342,  ...,     0,     0,     0],
        [  101,  9591, 33323,  ...,     0,     0,     0]])

In [18]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 전처리 - 테스트셋

In [19]:
sentences = test['상품설명']
print("sentences: ", len(sentences))
sentences[:10]

sentences:  7904


26218                               라이트그레이 골드 리본장식 탑지퍼 반지갑
25688                      rainbow ny pt 레드 히든밴딩 면혼방 여성 팬츠
10175                          다크그레이 솔리드 기본핏 울 혼방 스트레이트 팬츠
19866    on line limited summer run 블랙 남성 폴리 스판 스포츠 ice...
18983                        hartford women 카키 패턴 면 셔츠형원피스
19100                                     다크블루 면혼방 8부스키니팬츠
6562                                  alysi 옐로우 패턴 울혼방 가디건
8658                                      블랙 가죽 실버 로고 서류가방
1255                               아울렛new 화이트 이면배색 가죽 스니커즈
18833                                      옐로우 프린트 실크 랩스커트
Name: 상품설명, dtype: object

In [20]:
# BERT 입력 형식에 맞게 변환
sentences = _get_bert_sentences(test["상품설명"])

# 라벨 추출
labels = _get_label(test, '상품카테고리')

# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer, tokenized_texts = _get_tokenized_texts('bert-base-multilingual-cased', sentences)

# 토큰화된 문장 숫자로 변경
input_ids = _get_input_ids(128, tokenizer, tokenized_texts)

# 어텐션 마스크 얻기
attention_masks = _get_attention_masks(input_ids)

# 데이터를 파이토치의 텐서로 변환
test_inputs, test_labels, test_masks = _get_torch_tensor(input_ids, labels, attention_masks)

print("test_inputs: ", test_inputs.shape)
print("test_labels: ", test_labels.shape)
print("test_masks: ", test_masks.shape)

# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

test_inputs:  torch.Size([7904, 128])
test_labels:  torch.Size([7904])
test_masks:  torch.Size([7904, 128])


# 모델 생성

In [21]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [22]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [23]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(product_category))
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

![대체 텍스트](http://mccormickml.com/assets/BERT/padding_and_mask.png)

사전훈련된 BERT는 다양한 문제로 전이학습이 가능합니다. 여기서는 위의 그림과 같이 한 문장을 분류하는 방법을 사용합니다. 영화리뷰 문장이 입력으로 들어가면, 긍정/부정으로 구분합니다. 모델의 출력에서 [CLS] 위치인 첫 번째 토큰에 새로운 레이어를 붙여서 파인튜닝을 합니다. Huggning Face는 BertForSequenceClassification() 함수를 제공하기 때문에 쉽게 구현할 수 있습니다.
<br>
<br>
<br>

In [24]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# 모델 학습

In [25]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

# colab 튕김 방지

function ClickConnect() {var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); buttons.forEach(function(btn) { btn.click(); }); console.log("1분마다 자동 재연결"); document.querySelector("colab-toolbar-button#connect").click(); } setInterval(ClickConnect,1000*60);

https://somjang.tistory.com/entry/Google-Colab-%EC%9E%90%EC%A3%BC%EB%81%8A%EA%B8%B0%EB%8A%94-%EB%9F%B0%ED%83%80%EC%9E%84-%EB%B0%A9%EC%A7%80%ED%95%98%EA%B8%B0

In [26]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    667.    Elapsed: 0:05:49.

  Average training loss: 0.83
  Training epcoh took: 0:07:45

Running Validation...
  Accuracy: 0.9231
  Validation took: 0:00:18

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    667.    Elapsed: 0:05:48.

  Average training loss: 0.27
  Training epcoh took: 0:07:44

Running Validation...
  Accuracy: 0.9372
  Validation took: 0:00:18

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    667.    Elapsed: 0:05:48.

  Average training loss: 0.19
  Training epcoh took: 0:07:45

Running Validation...
  Accuracy: 0.9418
  Validation took: 0:00:18

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    667.    Elapsed: 0:05:48.

  Average training loss: 0.15
  Training epcoh took: 0:07:44

Running Validation...
  Accuracy: 0.9489
  Validation took: 0:00:18

Training complete!


In [27]:
# 모델 저장
# https://tutorials.pytorch.kr/beginner/saving_loading_models.html

model_PATH = "classification_with_HuggingFace_BERT/"

torch.save(model.state_dict(), model_PATH + "model_state_dict2.pt")
torch.save(optimizer.state_dict(), model_PATH + "optimizer_state_dict2.pt")

# 테스트셋 평가

In [28]:
def eval_model(device, model, test_dataloader):
    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(test_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("")
    print("Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("Test took: {:}".format(format_time(time.time() - t0)))

In [29]:
# test set 평가

eval_model(device, model, test_dataloader)

  Batch   100  of    247.    Elapsed: 0:00:26.
  Batch   200  of    247.    Elapsed: 0:00:50.

Accuracy: 0.9538
Test took: 0:01:01


In [31]:
# load model, optimizer 설정

model_PATH = "classification_with_HuggingFace_BERT/"
load_model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(product_category))
load_model.cuda()

# 옵티마이저 설정
load_optimizer = AdamW(load_model.parameters(),
                       lr = 2e-5, # 학습률,
                       eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                       )

load_model.load_state_dict(torch.load(model_PATH + "model_state_dict2.pt"))
load_optimizer.load_state_dict(torch.load(model_PATH + "optimizer_state_dict2.pt"))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [32]:
# load model 평가

eval_model(device, load_model, test_dataloader)

  Batch   100  of    247.    Elapsed: 0:00:25.
  Batch   200  of    247.    Elapsed: 0:00:50.

Accuracy: 0.9538
Test took: 0:01:01


# 새로운 문장 테스트

In [33]:
# 입력 데이터 변환
def convert_input_data(sentences):
    # list로 들어옴.
    sentences = ''.join(sentences) # list -> str
    sentences = _preprocessing(sentences) # text 전처리
    sentences = [sentences] # str -> array (list로 바꿔주는게 아니고 []를 사용해서 array로?..)

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print("tokenized_texts: ", tokenized_texts)

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [34]:
print("product category: ", (total_data["상품카테고리"].unique()))

product category:  ['지갑' '스웨터' '점퍼/아우터' '코트' '자켓' '팬츠' '신발' '티셔츠' '가방' '트렌치 코트' '모자' '드레스'
 '머플러/마스크' '홀더 및 케이스' '양말' '스카프' '블라우스' '셔츠' '넥타이' '베스트' '스커트' '장갑' '벨트'
 '주얼리 및 타이핀']


In [35]:
def predict_product_category(sent):
  logits = test_sentences([sent])
  predict_class = np.argmax(logits)
  print("sent: {0}, predict: {1}".format(sent, label_encoder.inverse_transform([predict_class])))

  return logits

In [38]:
# test

predict_product_category('크록스 클린 산타 크루즈 컷 로퍼 남성용 202972-4BM')

tokenized_texts:  [['크', '##록', '##스', '클', '##린', '산', '##타', '크', '##루', '##즈', '컷', '로', '##퍼', '남', '##성', '##용', '202', '##97', '##2', '4', '##b', '##m']]
sent: 크록스 클린 산타 크루즈 컷 로퍼 남성용 202972-4BM, predict: ['신발']


array([[ 0.32704595, -0.3970214 , -0.6286787 , -0.8869607 , -0.08011185,
         0.6154584 , -0.73269767, -0.90003073,  0.25708804, -1.4971191 ,
        -0.72188497, -0.8514829 ,  8.437163  , -0.13377562, -0.4716112 ,
        -0.42228216, -0.14737742, -0.9253256 , -0.40350467, -0.42210567,
        -0.8383144 ,  0.0645486 , -0.102325  , -0.6319899 ]],
      dtype=float32)

In [45]:
# Naver 쇼핑몰 데이터 분석

test_list = [
    '크록스 클린 산타 크루즈 컷 로퍼 남성용 202972-4BM', # 신발
    '뉴발란스 남녀공용 런닝화 MR530SH', # 신발
    '지오다노 남 테이퍼드 슬랙스 110511', # 바지
    '지오다노 남 MW 테이퍼드 링클프리 110901', # 바지
    '[아디다스]바시티 봄버 야구점퍼/GE1340', # 점퍼/아우터
    '[미니멀프로젝트] 미니멀프로젝트 피그먼트 시그니쳐 오버핏 후드집업 MZT106 / 4color w', # 점퍼/아우터
    '나이키 빅스우시 윈드 브레이커 바람막이 검흰 AR3132-010', # 점퍼/아우터
    '[다이애그널] [29EDITION]_Pretzel puff knit - ivory', # 스웨터
    '폴로 여성 울니트 Wool Vneck Sweater', # 스웨터
    '나인식스뉴욕 식스뉴욕 96ny 코튼 테리 루즈핏 풀오버 4종', # 스웨터
    '스튜디오톰보이 싱글 소매 리본 트랜치코트 9190111029 013', # 코트
    '로엠 변형 홑겹 트렌치코트 RMJTA2301A', # 코트
    '헨리코튼 코튼 린넨 혼방 피그먼트 다잉코트', # 코트
    '칼하트 킥플립 백팩 블랙', # 가방
    '루알리아 미니 피크닉 토트백', # 가방
    '라코스테 레이디 미니 크로스백 NF2609' # 가방
]

product_logits_list = []

for test_sent in test_list:
  product_logits = predict_product_category(test_sent)
  product_logits_list.append(product_logits.flatten()) # vector값 저장
  print()

tokenized_texts:  [['크', '##록', '##스', '클', '##린', '산', '##타', '크', '##루', '##즈', '컷', '로', '##퍼', '남', '##성', '##용', '202', '##97', '##2', '4', '##b', '##m']]
sent: 크록스 클린 산타 크루즈 컷 로퍼 남성용 202972-4BM, predict: ['신발']

tokenized_texts:  [['뉴', '##발', '##란', '##스', '남', '##녀', '##공', '##용', '런', '##닝', '##화', 'm', '##r', '##53', '##0', '##sh']]
sent: 뉴발란스 남녀공용 런닝화 MR530SH, predict: ['신발']

tokenized_texts:  [['지', '##오', '##다', '##노', '남', '테', '##이', '##퍼', '##드', '슬', '##랙', '##스', '110', '##51', '##1']]
sent: 지오다노 남 테이퍼드 슬랙스 110511, predict: ['팬츠']

tokenized_texts:  [['지', '##오', '##다', '##노', '남', 'm', '##w', '테', '##이', '##퍼', '##드', '링', '##클', '##프', '##리', '110', '##90', '##1']]
sent: 지오다노 남 MW 테이퍼드 링클프리 110901, predict: ['신발']

tokenized_texts:  [['아', '##디', '##다', '##스', '바', '##시', '##티', '봄', '##버', '야구', '##점', '##퍼', 'ge', '##13', '##40']]
sent: [아디다스]바시티 봄버 야구점퍼/GE1340, predict: ['점퍼/아우터']

tokenized_texts:  [['미', '##니', '##멀', '##프', '##로', '##젝', '##트', '미', '##니', '#

In [46]:
from sklearn.cluster import KMeans

corpus_embeddings = product_logits_list

num_clusters = 6
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(test_list[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['크록스 클린 산타 크루즈 컷 로퍼 남성용 202972-4BM', '뉴발란스 남녀공용 런닝화 MR530SH', '지오다노 남 MW 테이퍼드 링클프리 110901', '나이키 빅스우시 윈드 브레이커 바람막이 검흰 AR3132-010']

Cluster  2
['스튜디오톰보이 싱글 소매 리본 트랜치코트 9190111029 013', '로엠 변형 홑겹 트렌치코트 RMJTA2301A', '헨리코튼 코튼 린넨 혼방 피그먼트 다잉코트']

Cluster  3
['칼하트 킥플립 백팩 블랙', '루알리아 미니 피크닉 토트백', '라코스테 레이디 미니 크로스백 NF2609']

Cluster  4
['[미니멀프로젝트] 미니멀프로젝트 피그먼트 시그니쳐 오버핏 후드집업 MZT106 / 4color w', '[다이애그널] [29EDITION]_Pretzel puff knit - ivory', '폴로 여성 울니트 Wool Vneck Sweater']

Cluster  5
['지오다노 남 테이퍼드 슬랙스 110511']

Cluster  6
['[아디다스]바시티 봄버 야구점퍼/GE1340', '나인식스뉴욕 식스뉴욕 96ny 코튼 테리 루즈핏 풀오버 4종']

